In [1]:
import sys
sys.path.insert(0,'../scripts')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from joblib import dump
pd.set_option('max_column', None)
pd.set_option('display.float_format', '{:.2f}'.format)
%matplotlib inline

In [6]:
from load_data import loader
import sql_utils as sql
from Result_picker import picker

In [10]:
res = picker()

# Engagement score of each user

In [17]:
engagement_centroids = pd.read_csv('../data/engagement_centroid.csv')
engagement_centroids = engagement_centroids.iloc[:,1:]
engagement_centroids

,Bearer Id,Total Duration (hr),Total Data (MegaBytes)
0,145.79,106.46,144.66
1,1.03,1.16,1.03
2,-0.34,-0.41,-0.34
3,2.06,2.84,2.07
4,0.17,0.20,0.17


In [34]:
experience_centroids = pd.read_csv('../data/engagement_centroid.csv')
experience_centroids = experience_centroids.iloc[:,1:]
experience_centroids

,Bearer Id,Total Duration (hr),Total Data (MegaBytes)
0,145.79,106.46,144.66
1,1.03,1.16,1.03
2,-0.34,-0.41,-0.34
3,2.06,2.84,2.07
4,0.17,0.20,0.17


In [31]:
#standardization
engagement_metrics = pd.read_csv('../data/engagement_cluster.csv')
engagement_metrics.index = engagement_metrics['MSISDN/Number']
engagement_metrics = engagement_metrics.iloc[:,1:4]
engagement_metrics

,Bearer Id,Total Duration (hr),Total Data (MegaBytes)
MSISDN/Number,,,
33601001722.00,-0.12,-0.10,0.11
33601001754.00,-0.12,0.12,-0.32
33601002511.00,-0.12,-0.04,-0.06
33601007832.00,-0.12,-0.33,-0.16
33601008617.00,0.18,-0.38,0.45
...,...,...,...
33789996170.00,-0.12,-0.48,0.01
33789997247.00,-0.12,-0.02,-0.13
3197020876596.00,-0.12,2.52,-0.27


In [32]:
experience_metrics = pd.read_csv('../data/experience_cluster.csv')
experience_metrics.index = experience_metrics['MSISDN/Number']
experience_metrics = experience_metrics.iloc[:,1:4]
experience_metrics

,Avg Delay (ms),Avg Throughput (kbps),cluster
MSISDN/Number,,,
33601001722.00,-0.34,-0.54,1
33601001754.00,0.52,-0.54,2
33601002511.00,-0.23,-0.54,1
33601007832.00,-0.23,-0.53,1
33601008617.00,-0.23,-0.54,1
...,...,...,...
33789996170.00,-0.23,-0.53,1
33789997247.00,-0.64,-0.53,1
3197020876596.00,-0.23,-0.54,1


In [44]:
engagement_score = euclidean_distances(engagement_metrics.values.tolist(),[engagement_centroids.values.tolist()[0]])
engagement_score

array([[231.38738809],
       [231.55088615],
       [231.46246656],
       ...,
       [230.4293563 ],
       [231.27513087],
       [230.47554266]])

In [40]:
#Calculating the distance between the metric values and the centroid of the the first k-means cluster
experience_score = euclidean_distances(experience_metrics.values.tolist(),[experience_centroids.values.tolist()[0]])
experience_score

array([[231.1697678 ],
       [230.00832063],
       [231.10096659],
       ...,
       [231.10096659],
       [231.21656219],
       [231.53241121]])

In [41]:
# changing experience score from a list of lists to list of values only
experience_score = [val[0] for val in experience_score.tolist()]
experience_score[:2]

[231.1697677999321, 230.00832063158995]

# Satisfing score based on engagement and experience

In [46]:
#creating a dataframe to contain both engagement and experience score with index being 'MSISDN/Number' to identify the user
df = pd.DataFrame()
df['engagement_score'] = engagement_score
df['experience_score'] = experience_score
df.index = engagement_metrics.index
#Saving the data
results.add_data('engagement_score',df.iloc[:,:1])
results.add_data('experience_score',df.iloc[:,1:2])
#result
df

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series